# Table of Contents:
* [Import Packages](#first-bullet)
* [Load Voter File](#two-bullet)
* [Scrape Registered Voters By County](#three-bullet)
* [HuffPo Polls API](#four-bullet)
* [Follow The Money (Donations)](#five-bullet)

This notebook will take us through the process of importing the data to be used in the model.  Fist, we will load the voter  and  voter demographics files.  Next, we will scrape the NC Bureau of Elections website for the total number of registered voters by county and demographic group. Following that, we will use the Huffington Post polls API to pull in information on North Carolina.  Finally, we will pull in a CSV of donation data by district.

# Import Packages <a class="anchor" id="first-bullet"></a>

In [1]:
import pandas as pd
import numpy as np

In [6]:
import json
import requests
from bs4 import BeautifulSoup

In [7]:
from datetime import date, timedelta

In [8]:
from __future__ import print_function
import time
import pollster
from pollster.rest import ApiException
from pprint import pprint

from datetime import datetime, date, time, timedelta

# Load Voter File <a class="anchor" id="two-bullet"></a>

## Load Voter History File

The voter history file contains rounghly 30.6 million records.  It is a list of all active voters and each time they voted in an election since 2010.  Due to the size of the file, pulling it directly into a dataframe is not an option.  Instead, we will pull chunks of 80K rows at a time and pivot the data so that we only have one row per voter.  We only care about general elections, so we will be discarding rows in the table that are for local/ primary elections.  Because we are pulling in chunks of data, a voter may get split into two chunks, and then aggregated into two separate rows.  To ensure that we only have one row per voter, we will re-aggregate the data once we have pulled all 30.6MM records into our dataframe.

In [24]:
chunksize = 80_000 #10 ** 6
total = []
for chunk in pd.read_csv('./data/nc_boe_data/ncvhis_Statewide.txt', delimiter = '\t', encoding = "ISO-8859-1", chunksize=chunksize):
#for chunk in pd.read_csv('ncvhis1/ncvhis1.txt', delimiter = '\t', encoding = "ISO-8859-1", chunksize=chunksize):
    df_vot_hist = chunk[['county_id', 'election_desc', 'voting_method', 'voted_party_cd',
                  'pct_label', 'ncid', 'voted_county_id', 'vtd_label']]
    
    df_vot_hist['elec_plus_party'] = df_vot_hist['election_desc'] + df_vot_hist['voted_party_cd']
 

    df_vot_hist['gen_16_voted'] = [1 if x =='11/08/2016 GENERAL' else 0 for x in df_vot_hist['election_desc']] 
    df_vot_hist['gen_12_voted'] = [1 if x =='11/06/2012 GENERAL' else 0 for x in df_vot_hist['election_desc']] 
    df_vot_hist['gen_18_voted'] = [1 if x =='11/06/2018 GENERAL' else 0 for x in df_vot_hist['election_desc']] 
    df_vot_hist['gen_14_voted'] = [1 if x =='11/04/2014 GENERAL' else 0 for x in df_vot_hist['election_desc']] 
    df_vot_hist['gen_10_voted'] = [1 if x =='11/02/2010 GENERAL' else 0 for x in df_vot_hist['election_desc']] 

    df_vot_hist['gen_16_rep'] = [1 if x =='11/08/2016 GENERALREP' else 0 for x in df_vot_hist['elec_plus_party']]
    df_vot_hist['gen_12_rep'] = [1 if x =='11/06/2012 GENERALREP' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_18_rep'] = [1 if x =='11/06/2018 GENERALREP' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_14_rep'] = [1 if x =='11/04/2014 GENERALREP' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_10_rep'] = [1 if x =='11/02/2010 GENERALREP' else 0 for x in df_vot_hist['elec_plus_party']] 

    df_vot_hist['gen_16_dem'] = [1 if x =='11/08/2016 GENERALDEM' else 0 for x in df_vot_hist['elec_plus_party']]
    df_vot_hist['gen_12_dem'] = [1 if x =='11/06/2012 GENERALDEM' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_18_dem'] = [1 if x =='11/06/2018 GENERALDEM' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_14_dem'] = [1 if x =='11/04/2014 GENERALDEM' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_10_dem'] = [1 if x =='11/02/2010 GENERALDEM' else 0 for x in df_vot_hist['elec_plus_party']] 

    df_vot_hist['gen_16_ind'] = [1 if x =='11/08/2016 GENERALUNA' else 0 for x in df_vot_hist['elec_plus_party']]
    df_vot_hist['gen_12_ind'] = [1 if x =='11/06/2012 GENERALUNA' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_18_ind'] = [1 if x =='11/06/2018 GENERALUNA' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_14_ind'] = [1 if x =='11/04/2014 GENERALUNA' else 0 for x in df_vot_hist['elec_plus_party']] 
    df_vot_hist['gen_10_ind'] = [1 if x =='11/02/2010 GENERALUNA' else 0 for x in df_vot_hist['elec_plus_party']] 
    
    df_elect_hist = df_vot_hist[['ncid', 'gen_16_voted', 'gen_12_voted', 'gen_18_voted', 'gen_14_voted', 'gen_10_voted',
                            'gen_16_rep', 'gen_12_rep', 'gen_18_rep', 'gen_14_rep', 'gen_10_rep',
                            'gen_16_dem', 'gen_12_dem', 'gen_18_dem', 'gen_14_dem', 'gen_10_dem',
                            'gen_16_ind', 'gen_12_ind', 'gen_18_ind', 'gen_14_ind', 'gen_10_ind']].groupby(['ncid']).sum().reset_index()
    lol = df_elect_hist.values.tolist()
    total += lol
    print(len(lol))
print(len(total))

C:\Users\phill\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\phill\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\phill\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

15003
18251
28649
11326
13259
13333
17521
15816
15877
14203
11565
18288
14672
12885
16870
24569
18951
12895
13355
13745
12408
12705
13414
14661
15443
17116
20757
25431
24426
11223
11935
14660
11507
14641
19485
15388
18286
24791
22299
12701
13654
12802
14626
21636
16229
13336
16603
22713
13847
16101
19416
23422
10825
12904
12206
13546
21465
18022
15948
16393
11111
13554
17778
15134
18682
19559
21104
12925
14622
12068
13009
13866
15565
17489
19676
22519
29287
26382
9818
10015
18702
13910
19384
17688
26568
13096
13409
15030
12555
16351
14269
16587
10021
10912
11267
11405
12285
14358
14432
17331
19231
26530
28996
27312
9411
13544
20454
11799
12603
13352
14457
14510
15040
16231
17639
20628
22833
28859
35214
10876
11969
15436
20291
16771
19731
26432
20211
12838
14448
16743
12304
17214
17258
9671
9908
9779
9678
9737
10973
12132
12807
12945
14232
14748
15928
14383
13730
16114
18457
20961
21223
27784
29843
32390
9894
12385
20379
23930
13714
16463
16871
14947
21689
18708
21771
13006
13972
16782


In [25]:
cols = ['ncid', 'gen_16_voted', 'gen_12_voted', 'gen_18_voted', 'gen_14_voted', 'gen_10_voted',
                            'gen_16_rep', 'gen_12_rep', 'gen_18_rep', 'gen_14_rep', 'gen_10_rep',
                            'gen_16_dem', 'gen_12_dem', 'gen_18_dem', 'gen_14_dem', 'gen_10_dem',
                            'gen_16_ind', 'gen_12_ind', 'gen_18_ind', 'gen_14_ind', 'gen_10_ind']

In [26]:
df = pd.DataFrame(total,columns=cols)

In [27]:
df.head()

,ncid,gen_16_voted,gen_12_voted,gen_18_voted,gen_14_voted,gen_10_voted,gen_16_rep,gen_12_rep,gen_18_rep,gen_14_rep,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
0,AA100000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,AA100006,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,AA100007,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
3,AA100008,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,AA100009,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df.to_csv('./data/agged_voters.csv')

After performing our aggregation, we now have entries for 6,348,923 voters.  However, due to the way we pulled the data in (chunks), we may have two rows for certain voters, if their records were in two chunks.  We will perform a final aggregation to eliminate any potential duplicates.

In [29]:
df.shape

(6348923, 21)

In [30]:
df_elect_hist = df[['ncid', 'gen_16_voted', 'gen_12_voted', 'gen_18_voted', 'gen_14_voted', 'gen_10_voted',
                            'gen_16_rep', 'gen_12_rep', 'gen_18_rep', 'gen_14_rep', 'gen_10_rep',
                            'gen_16_dem', 'gen_12_dem', 'gen_18_dem', 'gen_14_dem', 'gen_10_dem',
                            'gen_16_ind', 'gen_12_ind', 'gen_18_ind', 'gen_14_ind', 'gen_10_ind']].groupby(['ncid']).sum().reset_index()

We now have slightly fewer voters, all unique. 

In [31]:
df_elect_hist.shape

(6348629, 21)

In [32]:
df_elect_hist.to_csv('./data/agged_dedupe_voters.csv')

In [33]:
df_elect_hist.head()

,ncid,gen_16_voted,gen_12_voted,gen_18_voted,gen_14_voted,gen_10_voted,gen_16_rep,gen_12_rep,gen_18_rep,gen_14_rep,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
0,AA100000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,AA100006,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,AA100007,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
3,AA100008,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,AA100009,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_elect_hist['ncid'] = df_elect_hist.index

In [2]:
df_elect_hist = pd.read_csv('./data/agged_dedupe_voters.csv')

In [4]:
df_elect_hist.head()

,Unnamed: 0,ncid,gen_16_voted,gen_12_voted,gen_18_voted,gen_14_voted,gen_10_voted,gen_16_rep,gen_12_rep,gen_18_rep,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
0,0,AA100000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,AA100006,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,AA100007,1,1,1,1,1,0,0,0,...,0,0,0,0,0,1,1,1,1,1
3,3,AA100008,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4,AA100009,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Load Voter Demo File

The next dataset we will pull in contains the voter ID and demographic information on all of the currently registered voters in North Carolina, from the BOE.  Because of the size of this dataset, we will pull it in as chunks, combining it all into one dataframe at the end.

In [5]:
import dask.dataframe as dd

In [3]:
voter_cols = ['county_id', 'county_desc', 'voter_reg_num', 'status_cd',
       'voter_status_desc', 'reason_cd', 'voter_status_reason_desc',
       'absent_ind', 'name_prefx_cd', 'last_name', 'first_name', 'middle_name',
       'name_suffix_lbl', 'res_street_address', 'res_city_desc', 'state_cd',
       'zip_code', 'mail_addr1', 'mail_addr2', 'mail_addr3', 'mail_addr4',
       'mail_city', 'mail_state', 'mail_zipcode', 'full_phone_number',
       'race_code', 'ethnic_code', 'party_cd', 'gender_code', 'birth_age',
       'birth_state', 'drivers_lic', 'registr_dt', 'precinct_abbrv',
       'precinct_desc', 'municipality_abbrv', 'municipality_desc',
       'ward_abbrv', 'ward_desc', 'cong_dist_abbrv', 'super_court_abbrv',
       'judic_dist_abbrv', 'nc_senate_abbrv', 'nc_house_abbrv',
       'county_commiss_abbrv', 'county_commiss_desc', 'township_abbrv',
       'township_desc', 'school_dist_abbrv', 'school_dist_desc',
       'fire_dist_abbrv', 'fire_dist_desc', 'water_dist_abbrv',
       'water_dist_desc', 'sewer_dist_abbrv', 'sewer_dist_desc',
       'sanit_dist_abbrv', 'sanit_dist_desc', 'rescue_dist_abbrv',
       'rescue_dist_desc', 'munic_dist_abbrv', 'munic_dist_desc',
       'dist_1_abbrv', 'dist_1_desc', 'dist_2_abbrv', 'dist_2_desc',
       'confidential_ind', 'birth_year', 'ncid', 'vtd_abbrv', 'vtd_desc']

In [6]:
chunksize = 100_000#5 #10 ** 6
total_demo = []
for chunk in pd.read_csv('./data/nc_boe_data/ncvoter_Statewide.txt', delimiter = '\t', encoding = "ISO-8859-1", chunksize=chunksize):
#for chunk in pd.read_csv('ncvhis1/ncvhis1.txt', delimiter = '\t', encoding = "ISO-8859-1", chunksize=chunksize):
    df_voters = chunk[voter_cols]
    df_voters = df_voters[['ncid', 'birth_year', 'nc_house_abbrv', 'nc_senate_abbrv', 'birth_age',
                          'race_code' ,'ethnic_code', 'registr_dt', 'party_cd', 'cong_dist_abbrv']]
    
    df_voters = df_voters.values.tolist()
    print(len(df_voters), 'iteration')
    total_demo += df_voters
    print(len(total_demo), 'total')

100000 iteration
100000 total


C:\Users\phill\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (33,35,40,41,44,45,47,48,49,60,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


100000 iteration
200000 total


C:\Users\phill\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (35,40,44,45,48,53,55,57,59,60,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


100000 iteration
300000 total


...

100000 iteration
7300000 total


C:\Users\phill\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (24,33,34,35,37,38,40,45,46,47,49,56,57,60,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


100000 iteration
7400000 total


C:\Users\phill\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (35,37,38,40,44,45,48,49,53,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


100000 iteration
7500000 total
15802 iteration
7515802 total


In [8]:
cols = ['ncid', 'birth_year', 'nc_house_abbrv', 'nc_senate_abbrv', 'birth_age',
                          'race_code' ,'ethnic_code', 'registr_dt', 'party_cd', 'cong_dist_abbrv']

df_voters = pd.DataFrame(total_demo, columns = cols)

In [9]:
df_voters.head()

,ncid,birth_year,nc_house_abbrv,nc_senate_abbrv,birth_age,race_code,ethnic_code,registr_dt,party_cd,cong_dist_abbrv
0,AA56273,1935,64.0,24.0,83,W,NL,10/01/1984,UNA,6.0
1,AA201627,1978,63.0,24.0,40,W,UN,02/23/2018,UNA,6.0
2,AA98377,1976,64.0,24.0,42,W,UN,03/26/1996,UNA,6.0
3,AA69747,1945,63.0,24.0,74,W,NL,08/15/1989,UNA,6.0
4,AA170513,1948,64.0,24.0,71,W,UN,03/07/2012,DEM,6.0


In [10]:
df_elect_hist.shape

(6348629, 22)

In [11]:
df_voters.shape

(7515802, 10)

## Combine Voter History and Voter Demographics 

Now that we have both voter history and election history imported and processed we will combine the two.  Due to the size of the datasets, we will be using Dask partitioned dataframes to avoid issues with memeory.

In [16]:
df = dd.concat([df_voters.set_index('ncid'),df_elect_hist.set_index('ncid')], axis=1, join='outer')

In [17]:
df.head()

,birth_year,nc_house_abbrv,nc_senate_abbrv,birth_age,race_code,ethnic_code,registr_dt,party_cd,cong_dist_abbrv,Unnamed: 0,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
AA100000,1967.0,64.0,24.0,51.0,W,NL,06/21/1996,UNA,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AA100004,1976.0,63.0,24.0,43.0,W,NL,06/13/1996,DEM,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA100006,1963.0,63.0,24.0,55.0,W,NL,06/14/1996,REP,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AA100007,1954.0,63.0,24.0,64.0,W,NL,06/11/1996,UNA,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
AA100008,1951.0,64.0,24.0,67.0,W,NL,06/14/1996,UNA,6.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
len(df)

7527956

In [22]:
import dask

In [25]:
#df.to_parquet('data/combined_vf.parquet', engine='pyarrow')

To read in the parquet file we created above, we will need to use the Pyarrow package, imported below.

In [2]:
import pyarrow

In [3]:
combined_vf = pd.read_parquet('combined_vf.parquet', engine='pyarrow')

In [4]:
combined_vf.head()

,birth_year,nc_house_abbrv,nc_senate_abbrv,birth_age,race_code,ethnic_code,registr_dt,party_cd,cong_dist_abbrv,Unnamed: 0,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
AA100000,1967.0,64.0,24.0,51.0,W,NL,06/21/1996,UNA,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AA100004,1976.0,63.0,24.0,43.0,W,NL,06/13/1996,DEM,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA100006,1963.0,63.0,24.0,55.0,W,NL,06/14/1996,REP,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AA100007,1954.0,63.0,24.0,64.0,W,NL,06/11/1996,UNA,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
AA100008,1951.0,64.0,24.0,67.0,W,NL,06/14/1996,UNA,6.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
combined_vf.shape

(7527956, 30)

In [6]:
combined_vf = combined_vf.fillna(value=0)

In [7]:
combined_vf['registr_dt'] = pd.to_datetime(combined_vf['registr_dt'],infer_datetime_format=True)

In [8]:
combined_vf.head()

,birth_year,nc_house_abbrv,nc_senate_abbrv,birth_age,race_code,ethnic_code,registr_dt,party_cd,cong_dist_abbrv,Unnamed: 0,...,gen_16_dem,gen_12_dem,gen_18_dem,gen_14_dem,gen_10_dem,gen_16_ind,gen_12_ind,gen_18_ind,gen_14_ind,gen_10_ind
AA100000,1967.0,64.0,24.0,51.0,W,NL,1996-06-21,UNA,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AA100004,1976.0,63.0,24.0,43.0,W,NL,1996-06-13,DEM,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AA100006,1963.0,63.0,24.0,55.0,W,NL,1996-06-14,REP,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AA100007,1954.0,63.0,24.0,64.0,W,NL,1996-06-11,UNA,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
AA100008,1951.0,64.0,24.0,67.0,W,NL,1996-06-14,UNA,6.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Calculate Participation Rates

For EDA, we will be creating a variety of graphs showing the participation rate of North Carolina voters.  We will need to calculate these from the combine voter file.  The below function will take in the election column we are summing up, the date of the election, the group we are looking to subset the voter file to, and finally the congressional district we are looking to subset to.

In [9]:
def overall_partic_rate(election_name, election_date, subset_column='all', subset='all', cd='all'):
    mask = combined_vf['registr_dt'] <= election_date # only count those who registration date makes them eligible
    # for all voters
    if subset_column == 'all':
        registered = len(combined_vf[mask])
        num_voted = combined_vf[[election_name]][mask].sum()
    # for looking at party, demo subset
    else:
        mask_2 = combined_vf[subset_column] == subset
        registered = len(combined_vf[mask & mask_2])
        num_voted = combined_vf[[election_name]][mask & mask_2].sum()  
    if cd != 'all':
        mask_3 = combined_vf['cong_dist_abbrv'] == cd
        registered = len(combined_vf[mask & mask_2 & mask_3])
        num_voted = combined_vf[[election_name]][mask & mask_2 & mask_3].sum()  
    else:
        pass
    return float(num_voted / registered)

Below is a list of all of participation rates we will calculate.  To use the participation rate function, we will thow this list into a for loop.

In [10]:
general_elctions = [('gen_18_voted', '2018-11-06'), 
                   ('gen_16_voted', '2016-11-08'),
                   ('gen_14_voted', '2014-11-04'),
                   ('gen_12_voted', '2012-11-06'),
                   ('gen_10_voted', '2010-11-02'),
# PARTY                    
                   ('gen_18_rep', '2018-11-06', 'party_cd', 'REP'), 
                   ('gen_16_rep', '2016-11-08', 'party_cd', 'REP'),
                   ('gen_14_rep', '2014-11-04', 'party_cd', 'REP'),
                   ('gen_12_rep', '2012-11-06', 'party_cd', 'REP'),
                   ('gen_10_rep', '2010-11-02', 'party_cd', 'REP'),
                   
                   ('gen_18_dem', '2018-11-06', 'party_cd', 'DEM'), 
                   ('gen_16_dem', '2016-11-08', 'party_cd', 'DEM'),
                   ('gen_14_dem', '2014-11-04', 'party_cd', 'DEM'),
                   ('gen_12_dem', '2012-11-06', 'party_cd', 'DEM'),
                   ('gen_10_dem', '2010-11-02', 'party_cd', 'DEM'),                   
# RACE
                   ('gen_18_voted', '2018-11-06', 'race_code', 'W'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'W'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'W'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'W'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'W'),                      

                   ('gen_18_voted', '2018-11-06', 'race_code', 'B'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'B'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'B'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'B'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'B'),

                   ('gen_18_voted', '2018-11-06', 'race_code', 'I'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'I'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'I'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'I'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'I'), 
                    
                   ('gen_18_voted', '2018-11-06', 'race_code', 'O'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'O'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'O'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'O'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'O'), 
                    
                   ('gen_18_voted', '2018-11-06', 'race_code', 'U'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'U'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'U'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'U'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'U'), 
                    
                   ('gen_18_voted', '2018-11-06', 'race_code', 'A'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'A'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'A'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'A'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'A'),                     
                    
                   ('gen_18_voted', '2018-11-06', 'race_code', 'M'), 
                   ('gen_16_voted', '2016-11-08', 'race_code', 'M'),
                   ('gen_14_voted', '2014-11-04', 'race_code', 'M'),
                   ('gen_12_voted', '2012-11-06', 'race_code', 'M'),
                   ('gen_10_voted', '2010-11-02', 'race_code', 'M'),
                    
# ethnic code
                   ('gen_18_voted', '2018-11-06', 'ethnic_code', 'NL'), 
                   ('gen_16_voted', '2016-11-08', 'ethnic_code', 'NL'),
                   ('gen_14_voted', '2014-11-04', 'ethnic_code', 'NL'),
                   ('gen_12_voted', '2012-11-06', 'ethnic_code', 'NL'),
                   ('gen_10_voted', '2010-11-02', 'ethnic_code', 'NL'),
                    
                   ('gen_18_voted', '2018-11-06', 'ethnic_code', 'UN'), 
                   ('gen_16_voted', '2016-11-08', 'ethnic_code', 'UN'),
                   ('gen_14_voted', '2014-11-04', 'ethnic_code', 'UN'),
                   ('gen_12_voted', '2012-11-06', 'ethnic_code', 'UN'),
                   ('gen_10_voted', '2010-11-02', 'ethnic_code', 'UN'),  
                    
                   ('gen_18_voted', '2018-11-06', 'ethnic_code', 'HL'), 
                   ('gen_16_voted', '2016-11-08', 'ethnic_code', 'HL'),
                   ('gen_14_voted', '2014-11-04', 'ethnic_code', 'HL'),
                   ('gen_12_voted', '2012-11-06', 'ethnic_code', 'HL'),
                   ('gen_10_voted', '2010-11-02', 'ethnic_code', 'HL'),  
                   ]
gen_participation = pd.DataFrame(columns=['election_date', 'category', 'segment', 'turnout_rate'])
for i in general_elctions:
    election_name = i[0]
    election_date = i[1]
    try:   # if it is an oveall entry, this index will not exist 
        subset_column = i[2]
    except:
        subset_column = 'all'
    try:
        subset = i[3]
    except:
        subset = 'all'
    particip = overall_partic_rate(election_name, election_date, subset_column, subset)
    # set election name if there is a subset name
    #gen_overall_participation.update( {dictionary_key : particip} )
    gen_participation = gen_participation.append({'election_date': election_date, 'category': subset_column,
                                                  'segment': subset, 'turnout_rate' : particip}, ignore_index=True)

In [11]:
gen_participation.shape

(65, 4)

In [12]:
gen_participation.to_csv('gen_participation_full_dataset.csv')

To calculate the participation rates by age group, we will loop through all of the disctince ages in the voter file, calculating participation rate for eeach election.  Because the voter file was accessed in 2019, all of the ages for the entries are their age as of 2019.  To calculate the age that they were when each election happened, we will calculate a column named "real_age" which is takes the differene between 2019 and the year of the election.

In [13]:
general_elctions = [('gen_18_voted', '2018-11-06'), 
                   ('gen_16_voted', '2016-11-08'),
                   ('gen_14_voted', '2014-11-04'),
                   ('gen_12_voted', '2012-11-06'),
                   ('gen_10_voted', '2010-11-02')]
# PARTY              
for i in range(18,int(max(combined_vf['birth_age']) + 1)):
    age = i
    age_real = i
    entry = ('gen_18_voted','2018-11-06','birth_age',age, age_real)
    general_elctions.append(entry)

for i in range(18,int(max(combined_vf['birth_age']) + 1)):
    age = i  
    age_real = i - 2
    entry = ('gen_16_voted','2016-11-08','birth_age',age, age_real)
    general_elctions.append(entry)
    
for i in range(18,int(max(combined_vf['birth_age']) + 1)):
    age = i 
    age_real = i - 4
    entry = ('gen_14_voted','2014-11-04','birth_age',age, age_real)
    general_elctions.append(entry)

for i in range(18,int(max(combined_vf['birth_age']) + 1)):
    age = i 
    age_real = i - 6
    entry = ('gen_12_voted','2012-11-06','birth_age',age, age_real)
    general_elctions.append(entry)
    
for i in range(18,int(max(combined_vf['birth_age']) + 1)):
    age = i 
    age_real = i - 8
    entry = ('gen_10_voted','2010-11-02','birth_age',age, age_real)
    general_elctions.append(entry)
    
gen_participation_age = pd.DataFrame(columns=['election_date', 'category', 'segment', 'turnout_rate'])
for i in general_elctions:
    election_name = i[0]
    election_date = i[1]
    try:   # if it is an oveall entry, this index will not exist 
        subset_column = i[2]
    except:
        subset_column = 'all'
    try:
        subset = i[3]
        subset_real = i[4]
    except:
        subset = 'all'
        subset_real = 'all'
   # print(election_name,election_date, subset_column, subset, subset_real )
    particip = overall_partic_rate(election_name, election_date, subset_column, subset)
    # set election name if there is a subset name
    #gen_overall_participation.update( {dictionary_key : particip} )
    if subset == 'all' or subset_real >= 18:
        gen_participation_age = gen_participation_age.append({'election_date': election_date, 'category': subset_column,
                                                  'segment': subset_real, 'turnout_rate' : particip}, ignore_index=True)
    else:
        pass

In [14]:
gen_participation_age.shape

(1115, 4)

In [15]:
gen_participation_age.to_csv('gen_participation_age_full_dataset.csv')

Finally, we will calculate the participation by congressional group and party.

In [16]:
general_elections = []

gen_participation_cd = pd.DataFrame(columns=['election_date', 'party', 'cd', 'turnout_rate'])

for i in range(0,int(max(combined_vf['cong_dist_abbrv']) + 1)):
    list_of_elections = [('gen_18_voted', '2018-11-06', 'party_cd', 'REP', i), 
                         ('gen_16_voted', '2016-11-08', 'party_cd', 'REP', i),
                         ('gen_14_voted', '2014-11-04', 'party_cd', 'REP', i),
                         ('gen_12_voted', '2012-11-06', 'party_cd', 'REP', i),
                         ('gen_10_voted', '2010-11-02', 'party_cd', 'REP', i),

                         ('gen_18_voted', '2018-11-06', 'party_cd', 'DEM', i), 
                         ('gen_16_voted', '2016-11-08', 'party_cd', 'DEM', i),
                         ('gen_14_voted', '2014-11-04', 'party_cd', 'DEM', i),
                         ('gen_12_voted', '2012-11-06', 'party_cd', 'DEM', i),
                         ('gen_10_voted', '2010-11-02', 'party_cd', 'DEM', i)
                        ]
    general_elections.append(list_of_elections)


    
for i in general_elections:
    for j in i:
        election_name = j[0]
        election_date = j[1]
        subset = j[2]
        subset_column = j[3]
        cd = j[4]
      #  print(election_name,election_date, subset, subset, subset_column, cd )
        particip = overall_partic_rate(election_name, election_date, subset, subset_column, cd)
     #   print(particip)
        # set election name if there is a subset name
        #gen_overall_participation.update( {dictionary_key : particip} )
        gen_participation_cd = gen_participation_cd.append({'election_date': election_date, 
                                                      'party': subset_column, 'cd': cd, 'turnout_rate' : particip}, ignore_index=True)

In [17]:
gen_participation_cd.to_csv('gen_participation_cd_full_dataset.csv')

# Scrape Registered Voters By County <a class="anchor" id="three-bullet"></a>

Another plot we will build for our EDA will show how many voters are registered in each county, by month, by group.

We will scrape this data from the BOE website. It is updated each Saturday, so we will have to find the dates for all Saturdays for each of the years in question, and then use those as the URLs for each of our scrapes.

In [25]:
# modified from https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python
def allsaturdays(year):
  #  print(year)
   # print(date(year, 1, 1))
    d = date(year, 1, 1)
  #  print(d.weekday())
    if d.weekday() == 6: # if jan 1 is a sat, make the first sunday 6 days after jan 1
        d += timedelta(6)
      #  print(d)
    else:    
        d += timedelta(days = 5 - d.weekday() + 7 % 7)  # First sat
      #  print(d)

    while d.year == year:
        yield d
        d += timedelta(days = 7)

In [26]:
# function to create list of urls to scrape, each saturday of the year.  will have to run this for each year and add them all
# to one list
def add_sats_in_year_to_list(year):
    sats = []
    urls = []
    for d in allsaturdays(year):
        sats.append(d)
        all_sats.append(d)
    for i in sats:
        url = 'https://vt.ncsbe.gov/RegStat/Results/?date=' + str(i.month).zfill(2) + '%2F' + str(i.day).zfill(2) + '%2F' + str(i.year)
        #urls.append(url)
        all_urls.append(url)
    print(len(sats))

In [27]:
all_urls = []
all_sats = []

In [28]:
#list_of_years = [2010]
list_of_years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
for year in list_of_years:
    print(year)
    add_sats_in_year_to_list(year)

2009
52
2010
52
2011
53
2012
52
2013
52
2014
52
2015
52
2016
53
2017
52
2018
52
2019
52


In [29]:
columns_df = ["CountyName","Democrats","Republicans","Libertarians","Green","Constitution","Unaffiliated","White","Black","AmericanIndian"
,"Other","Hispanic","Male","Female","UnDisclosedGender","Total", "Date"]

In [30]:
voter_reg_df = pd.DataFrame(columns=columns_df)

In [31]:
all_years_data = []
all_dates = []
for idx_date, url in enumerate(all_urls):
    res = requests.get(url)
    date = all_sats[idx_date]
# make sure status code is 200, else error
    res.status_code

    soup = BeautifulSoup(res.content, 'lxml')

    table_data = soup.text.split('var data = ')[1].split('\r\n\t\t\t//')[0]

    rows = table_data.split('{')

    all_data = []
    for  i in rows[1:]:
      #  print(all_sats[idx])
       # break
        
        r2 = i.split('":')
        r2_3 = [i.split(",") for i in r2]
        row_data = []
        for idx,  i in enumerate(r2_3[1:]):
            if idx == 0:
                clean = i[0].replace('}',"").replace(']',"")
            else:
                clean = int(i[0].replace('}',"").replace(']',""))
            row_data.append(clean)
           # print(clean)
            
            #print(row_data)
        row_data.append(date)
       # print(row_data)
        all_data.append(row_data)
        #print(all_data)
    all_years_data.append(all_data)#(all_data)
    #print(all_years_data)
#all_years_data

In [32]:
for i in all_years_data:
    #print(i)
   # print(columns_df)
    df = pd.DataFrame(i, columns=columns_df)
   # df['date'] = all_sats[0][idx]
    voter_reg_df = pd.concat([voter_reg_df, df])

In [33]:
voter_reg_df.head()

,CountyName,Democrats,Republicans,Libertarians,Green,Constitution,Unaffiliated,White,Black,AmericanIndian,Other,Hispanic,Male,Female,UnDisclosedGender,Total,Date
0,"""ALAMANCE""",41349,30469,62,0,0,18164,68769,17838,176,1175,939,40062,49579,0,90044,2009-01-03
1,"""ALEXANDER""",8565,10841,25,0,0,5042,22982,1114,28,151,125,11575,12777,0,24473,2009-01-03
2,"""ALLEGHANY""",3609,2285,5,0,0,1379,7072,79,9,36,60,3404,3840,0,7278,2009-01-03
3,"""ANSON""",12376,2331,4,0,0,1848,8375,7864,37,64,21,7282,9052,0,16559,2009-01-03
4,"""ASHE""",7696,8721,12,0,0,4070,20150,107,11,89,76,9583,10876,0,20499,2009-01-03


In [ ]:
#voter_reg_df.to_csv('./data/nc_boe_data/voter_reg_by_week_2008_2019_all.csv')

In [34]:
list_of_reg_cols = list(voter_reg_df.columns)

In [35]:
list_of_reg_cols = [e for e in list_of_reg_cols if e not in ('Unnamed: 0','CountyName', 'Date' )]

In [36]:
list_of_reg_cols

['Democrats',
 'Republicans',
 'Libertarians',
 'Green',
 'Constitution',
 'Unaffiliated',
 'White',
 'Black',
 'AmericanIndian',
 'Other',
 'Hispanic',
 'Male',
 'Female',
 'UnDisclosedGender',
 'Total']

In [37]:
#reg_by_seg = pd.DataFrame(columns=['Total', 'seg'])

for i in list_of_reg_cols:
    print(i)
    # first round, make df, then subsequent rounds add to it
    if i == 'Democrats':
        df = voter_reg_df[[i, 'Date', 'CountyName']].groupby(['Date', 'CountyName']).sum()
        df['seg'] = i
        reg_by_seg = df.rename(columns={ df.columns[0]: "Total" })
       # print(reg_by_seg.head())
    else:
        df = voter_reg_df[[i, 'Date', 'CountyName']].groupby(['Date', 'CountyName']).sum()
        df['seg'] = i
        df = df.rename(columns={ df.columns[0]: "Total" })
       # print(df.head())
        reg_by_seg = pd.concat([df, reg_by_seg])
       # print(reg_by_seg)

Democrats
Republicans
Libertarians
Green
Constitution
Unaffiliated
White
Black
AmericanIndian
Other
Hispanic
Male
Female
UnDisclosedGender
Total


In [38]:
reg_by_seg = reg_by_seg.reset_index(level=[ 'CountyName'])

In [39]:
reg_by_seg.index = pd.to_datetime(reg_by_seg.index)

In [40]:
#reg_by_seg.resample('M').mean()
reg_by_seg_mo = reg_by_seg.groupby([pd.TimeGrouper(freq='M'), 'CountyName', 'seg'])['Total'].mean()

C:\Users\phill\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


In [41]:
group_reg = pd.DataFrame(reg_by_seg_mo)

In [42]:
group_reg = group_reg.reset_index(level=['Date', 'CountyName', 'seg'])

In [43]:
group_reg.to_csv('./data/grouped_registration.csv')

# HuffPo Polls API <a class="anchor" id="four-bullet"></a>

As an additional source of data, we will pull in President Obama's polling numbers for North Carolina, by week.

In [44]:
#https://github.com/huffpostdata/python-pollster/blob/master/example.py
api = pollster.Api()
nc_obama_job_approval_poll = api.charts_slug_pollster_trendlines_tsv_get('north-carolina-obama-job-approval')

C:\Users\phill\Anaconda3\lib\site-packages\pollster\api_client.py:164: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return_data = pandas.read_table(StringIO(response_data.data), **kwargs)


In [45]:
nc_obama_job_approval_poll.head()

,label,date,value,low,high
0,Approve,2009-01-18,55.9701,NaN,NaN
1,Approve,2009-01-22,55.8394,NaN,NaN
2,Approve,2009-02-15,55.0659,NaN,NaN
3,Approve,2009-02-19,54.9388,NaN,NaN
4,Approve,2009-02-26,54.7175,NaN,NaN


In [46]:
nc_obama_job_approval_poll.set_index('date', inplace=True)

In [47]:
c = nc_obama_job_approval_poll.index.to_series().between('2009-01-18', '2009-01-18')
nc_obama_job_approval_poll[c]

,label,value,low,high
date,,,,
2009-01-18,Approve,55.9701,NaN,NaN
2009-01-18,Disapprove,32.8340,NaN,NaN
2009-01-18,Undecided,9.6049,NaN,NaN


In [50]:
nc_obama_job_approval_poll.to_csv('./data/nc_obama_job_approval_poll.csv')

# Follow The Money (Donations) <a class="anchor" id="five-bullet"></a>

To bring in donation data, we will be using a CSV that was created manually from the tables on Follow the Money.  Scraping this data was not an option.

In [51]:
# data from https://www.followthemoney.org/tools/election-overview?s=NC&y=2010
df_money = pd.read_csv('./data/follow_the_money/donations.csv')

In [52]:
df_money.head()

,Candidate,Election Status,Status of Candidate,Specific Party,General Party,Election Jurisdiction,Election Year,Election Type,Office Sought,Incumbency Status,# of Records,Total $
0,"BUTTERFIELD JR, GEORGE KENNETH (GK)",WON-GENERAL,WON,DEMOCRATIC,DEMOCRATIC,NC,2010,STANDARD,US HOUSE DISTRICT NC 001,INCUMBENT,724.0,824065.0
1,"CARTER, JOHN",LOST-PRIMARY,LOST,REPUBLICAN,REPUBLICAN,NC,2010,STANDARD,US HOUSE DISTRICT NC 001,OPEN,0.0,0.0
2,"GRIMES, JERRY",LOST-PRIMARY,LOST,REPUBLICAN,REPUBLICAN,NC,2010,STANDARD,US HOUSE DISTRICT NC 001,OPEN,72.0,11838.0
3,"LARKINS, CHAD",LOST-PRIMARY,LOST,DEMOCRATIC,DEMOCRATIC,NC,2010,STANDARD,US HOUSE DISTRICT NC 001,CHALLENGER,3.0,450.0
4,"MILLER, JIM",LOST-PRIMARY,LOST,REPUBLICAN,REPUBLICAN,NC,2010,STANDARD,US HOUSE DISTRICT NC 001,OPEN,0.0,0.0


In [53]:
df_money.groupby(['Office Sought','Election Year',  'General Party' ]).agg({'Total $':'sum','# of Records':'sum'}).reset_index().head()

,Office Sought,Election Year,General Party,Total $,# of Records
0,US HOUSE DISTRICT NC 001,2010,DEMOCRATIC,824515.0,727.0
1,US HOUSE DISTRICT NC 001,2010,REPUBLICAN,144787.0,256.0
2,US HOUSE DISTRICT NC 001,2012,DEMOCRATIC,899936.0,889.0
3,US HOUSE DISTRICT NC 001,2012,REPUBLICAN,0.0,0.0
4,US HOUSE DISTRICT NC 001,2012,THIRD-PARTY,0.0,0.0


In [54]:
df_money.groupby(['Election Year',  'General Party' ]).agg({'Total $':'sum','# of Records':'sum'}).reset_index()

,Election Year,General Party,Total $,# of Records
0,2010,DEMOCRATIC,13453750.0,17107.0
1,2010,REPUBLICAN,24200635.0,30308.0
2,2010,THIRD-PARTY,29966.0,54.0
3,2012,DEMOCRATIC,9067600.0,11597.0
4,2012,REPUBLICAN,19998004.0,19510.0
5,2012,THIRD-PARTY,4466.0,46.0
6,2014,DEMOCRATIC,29092380.0,49591.0
7,2014,REPUBLICAN,25525712.0,28690.0
8,2014,THIRD-PARTY,39261.0,68.0
9,2016,DEMOCRATIC,18246630.0,36513.0
